# MNIST convolutional neural networks with slim

* MNIST data를 가지고 **convolutional neural networks**를 만들어보자.
  * [참고: TensorFlow.org](https://www.tensorflow.org/get_started/mnist/pros)
  * [소스: mnist_deep.py](https://github.com/tensorflow/tensorflow/blob/r1.4/tensorflow/examples/tutorials/mnist/mnist_deep.py)를 slim을 이용하여 수정함
  * [`tf.contrib.slim` 참고](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim)

### Import modules

In [ ]:
"""A deep MNIST classifier using convolutional layers.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/pros
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

slim = tf.contrib.slim

tf.set_random_seed(219)

### Import data

In [ ]:
data_dir = '../mnist'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

### Show the MNIST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

index = 999
print("label = ", np.argmax(mnist.train.labels[index]))
plt.imshow(mnist.train.images[index].reshape(28, 28), cmap='gray')
plt.show()

### Create the model

* [`tf.contrib.layers`](https://www.tensorflow.org/api_guides/python/contrib.layers)
* [`tf.contrib.layers.conv2d`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/conv2d)
* [`tf.contrib.layers.max_pool2d`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/max_pool2d)

In [ ]:
def deepnn_slim(x):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  h_conv1 = slim.conv2d(x_image, 32, [5, 5], scope='conv1')

  # Pooling layer - downsamples by 2X.
  h_pool1 = slim.max_pool2d(h_conv1, [2, 2], scope='pool1')

  # Second convolutional layer -- maps 32 feature maps to 64.
  h_conv2 = slim.conv2d(h_pool1, 64, [5, 5], scope='conv2')

  # Second pooling layer.
  h_pool2 = slim.max_pool2d(h_conv2, [2, 2], scope='pool2')

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  h_pool2_flat = slim.flatten(h_pool2, scope='flatten')
  h_fc1 = slim.fully_connected(h_pool2_flat, 1024, scope='fc1')

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  keep_prob = tf.placeholder(tf.float32)
  h_fc1_drop = slim.dropout(h_fc1, keep_prob, scope='dropout')

  # Map the 1024 features to 10 classes, one for each digit
  y_conv = slim.fully_connected(h_fc1_drop, 10, activation_fn=None, scope='fc2')
    
  return y_conv, keep_prob

### Create the placeholder

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

### Build the model and define loss and optimizer

* [`tf.losses.softmax_cross_entropy`](https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy)

In [ ]:
# Build the graph for the deep net
y_conv, keep_prob = deepnn_slim(x)

#with tf.name_scope('loss'):
#  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
#                                                          logits=y_conv)
#cross_entropy = tf.reduce_mean(cross_entropy)
cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=y_,
                                                logits=y_conv,
                                                scope='loss')

with tf.name_scope('adam_optimizer'):
  train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope('accuracy'):
  correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
  correct_prediction = tf.cast(correct_prediction, tf.float32)
accuracy = tf.reduce_mean(correct_prediction)

graph_location = 'graphs/code12_mnist_slim'
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())

### tf.Session() and train

In [ ]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())

max_step = 100
for step in range(max_step+1):
  batch = mnist.train.next_batch(32)
  feed_dict_train = {x: batch[0],
                     y_: batch[1],
                     keep_prob: 0.5}
  feed_dict_eval = {x: batch[0],
                    y_: batch[1],
                    keep_prob: 1.0}
  _, ce = sess.run([train_step, cross_entropy], feed_dict=feed_dict_train)
  if step % 10 == 0:
    train_accuracy = sess.run(accuracy, feed_dict=feed_dict_eval)
    print('step %d, training accuracy %g, cross_entropy %g' % (step, train_accuracy, ce))

### Test trained model

* test accuracy: 0.8308

In [ ]:
print('test accuracy %g' % sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

### test images inference

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

test_batch_size = 16
batch_xs, _ = mnist.test.next_batch(test_batch_size)
y_pred = sess.run(y_conv, feed_dict={x: batch_xs, keep_prob: 1.0})

fig = plt.figure(figsize=(16, 10))
for i, (px, py) in enumerate(zip(batch_xs, y_pred)):
  p = fig.add_subplot(4, 8, i+1)
  p.set_title("y_pred: {}".format(np.argmax(py)))
  p.imshow(px.reshape(28, 28), cmap='gray')
  p.axis('off')

## 직접 실습

* 여러가지 hyper-parameter들을 바꿔가면서 accuracy를 높혀보자